# Dust-line Mass calibration

The surface density estimate after Powell et al. 2017 but accounting properly for pre-factors is

$$\Sigma_\mathsf{g} = \frac{\gamma}{2} \frac{t_\mathsf{disk}\, v_0 \, \rho_\mathsf{s} \, \lambda}{r}$$

## Setup

In [ ]:
from pathlib import Path

import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import corner

import dipsy
import estimator
import analysis_mass

M_sun = dipsy.cgs_constants.M_sun
year = dipsy.cgs_constants.year
au = dipsy.cgs_constants.au
year = dipsy.cgs_constants.year
lams = analysis_mass.lams

Define constants

In [ ]:
flux_fraction = 0.68

In [ ]:
fname = 'dustlines.hdf5'

In [ ]:
t_disk = 3e6 * year

## Load a simulation

Get number of simulations

In [ ]:
with h5py.File(fname, 'r') as fh:
    n_sim = len(fh)

**Pick a random model**

In [ ]:
key = f'{int(n_sim * np.random.rand()):07d}'
# key = '0006370'
print(key)

**load the simulation**

In [ ]:
# actual simulation
sim = dipsy.utils.read_from_hdf5(fname, key)
it = sim['time'].searchsorted(t_disk)

## Fitting for dust lines

### Fitting using the function

In [ ]:
settings = {
    'time': t_disk,    
    'q' : 3.5,
    'flux_fraction': 0.68,
    'fname_in' : fname,
    'fname_out' : 'test.hdf5',
    'opac' : dipsy.Opacity(input='ricci_compact.npz'),
    'fct_nr' : 3,
}

In [ ]:
import analysis_mass

In [ ]:
res = analysis_mass.parallel_analyze(key, settings=settings, debug=True, progress=True, n_burnin=400, n_steps=1500)

### Test the parameter guess

In [ ]:
from importlib import reload
reload(estimator)

In [ ]:
obs = dipsy.get_observables(
    sim['r'],
    sim['sig_g'][it],
    sim['sig_d'][it],
    sim['a_max'][it],
    sim['T'][it],
    settings['opac'],
    lams,
)

In [ ]:
dipsy.fortran.crop=1e-10
ilam = 2

x = sim['r'] / au
y = obs.I_nu[ilam]
mask = x>1
x = x[mask]
y = y[mask]
p_guess, di = estimator.guess(x, y, 10, debug=True)

In [ ]:
f, ax = plt.subplots(2, 1, sharex=True, figsize=(8,8))

ax[0].loglog(x, y, 'k', lw=2)
for _p in p_guess:
    ax[0].loglog(x , dipsy.fortran.pwr2_logit(_p, x), 'g', alpha=0.5, lw=1)
    print('{:.2f}'.format(dipsy.fortran.lnp_pwr2_logit(_p, x, y)))

ax[1].semilogx(di['x'], di['exponent2'])

ax[1].axvline(di['r_dust'], c='r', lw=1, label=r'$r_\mathrm{dust}$')
ax[1].axvline(di['r_out'], c='k', lw=2, label=r'$r_\mathrm{out}$')
for _r in di['r_list']:
    ax[1].axvline(_r, c='k', lw=1, ls=':')
    
ax[1].legend()
ax[1].set_xlim(left=1);
ax[0].set_ylim(dipsy.fortran.crop, 1e4);

## Analyze results of the fitting

In [ ]:
print(f"Mass off by factor of {res['M_est']/res['M_gas']:.2g}")

x = sim['r'] / au

fig = plt.figure(constrained_layout=True, figsize=(12, 5))
gs = fig.add_gridspec(4, 4)
ax = fig.add_subplot(gs[:4, :2])
ax.set_xlabel(r'$r$ [au]')
ax.set_ylabel(r'$I_\nu$ [erg/(s cm$^2$ Hz sr)]')

for ilam in np.arange(len(lams)):
    
    # find the best match
    sampler = res['samplers'][ilam]
    discard = res['discards'][ilam]
    slice = sampler.lnprobability[:, discard:]
    idx = np.unravel_index(slice.argmax(), slice.shape)
    ln_best = slice[idx[0], idx[1]]
    p_best  = sampler.chain[:, discard:, :][idx[0], idx[1], :]
    _r_best = p_best[-1]
    print(f'r_best = {_r_best:.2g} au')
    
    txt = f'$\lambda = {lams[ilam] * 1e4:.0f}$ micron'
    
    # plot the model and determined dust line
    
    line, = ax.loglog(x, obs.I_nu[ilam], label=txt)
    ax.axvline(_r_best, c=line.get_color(), ls=':')
    
    # plot the logp evolution
    
    col = ilam//2
    row = ilam%2
    ax2 = fig.add_subplot(gs[2+row, 2+col])
    ax2.semilogy(-sampler.lnprobability.T, c='k', alpha=0.3);
    ax2.set_ylim(top=1e5)
    ax2.set_title(txt, {'color':line.get_color()})
    ax2.set_xlabel('iteration #')
    ax2.set_ylabel(r'$-\log P$')
    
    # overplot the fit
    
    if settings['fct_nr'] == 3:
        ym = dipsy.fortran.pwr2_logit(p_best, x)
    elif settings['fct_nr'] == 1:
        ym = dipsy.fortran.pwr1(p_best, x)
    ax.loglog(x, ym, c=line.get_color(), ls='--')
    

ax.legend()
ax.set_xlim(.05, 1e3)
ax.set_ylim(dipsy.fortran.crop, 1e2);

**Dust and gas surface densities from simulation and Powell method**

In [ ]:
fig, ax = plt.subplots()
ax.loglog(res['r_dust'], res['sig_g'], 'kx', label='Powell method')
ax.loglog(sim['r'] / au, sim['sig_g'][it], '-', label='simulation, gas')
ax.loglog(sim['r'] / au, sim['sig_d'][it], '-', label='simulation, dust')
ax.legend()
ax.set_ylim(1e-7, 1e4)
ax.set_xlabel(r'$r$ [au]')
ax.set_ylabel(r'$\Sigma$ [g/cm$^2$]');